#  Part2. 종목별 주가 정보

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

https://hogni.tistory.com/52 <br>
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201

In [9]:
# 상장회사 목록
df_code = pd.read_csv('data.csv',\
                  encoding='euc-kr')
df_code = df_code[['한글 종목명','한글 종목약명','단축코드']]
df_code

,한글 종목명,한글 종목약명,단축코드
0,(주)마이크로컨텍솔루션,마이크로컨텍솔,098120
1,(주)스카이이앤엠,스카이이앤엠,131100
2,(주)포스코엠텍,포스코엠텍,009520
3,AJ네트웍스보통주,AJ네트웍스,095570
4,AK홀딩스보통주,AK홀딩스,006840
5,BGF리테일보통주,BGF리테일,282330
6,BGF보통주,BGF,027410
7,BNK금융지주보통주,BNK금융지주,138930
8,BYC1우선주,BYC우,001465
9,BYC보통주,BYC,001460


In [11]:
def reverse_rolling(series, windows_):

    # 시리즈 인덱스 저장하기
    index = series.index
    # 컬럼 뒤집기
    series = pd.DataFrame(series.iloc[::-1])
    # 컬럼뒤집어서 이동평균
    series = series.rolling(window = int(windows_)).mean() 
    # 컬럼 다시 뒤집기
    series = series.iloc[::-1]
    # 저장했던 index 다시 집어넣기
    series['index'] = index
    # index 재설정하기
    series = series.set_index('index')


    return series

In [54]:
# 종목코드 기준으로 df별로 돌릴거

def cross_score(df):

    df['ma5']=reverse_rolling(df['주식_종가'],5).fillna(value=0)
    df['ma5_temp'] = df['ma5'].shift(fill_value=0).fillna(value=0)
    df['ma5_slope'] = pd.to_numeric(((df['ma5_temp']-df['ma5']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)
# pd.to_numeric(df['ma5_slope']) str->int

## 20일 기준 이동평균
    df['ma20']=reverse_rolling(df['주식_종가'],20).fillna(value=0)
    df['ma20_temp'] = df['ma20'].shift(fill_value=0).fillna(value=0)
    df['ma20_slope']=pd.to_numeric(((df['ma20_temp']-df['ma20']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)

## 60일 기준 이동평균
    df['ma60']=reverse_rolling(df['주식_종가'],60).fillna(value=0)
    df['ma60_temp'] = df['ma60'].shift(fill_value=0).fillna(value=0)
    df['ma60_slope'] = pd.to_numeric(((df['ma60_temp']-df['ma60']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)

## 120일 기준 이동평균
    df['ma120']=reverse_rolling(df['주식_종가'],120).fillna(value=0)
    df['ma120_temp'] = df['ma120'].shift(fill_value=0).fillna(value=0)
    df['ma120_slope']=pd.to_numeric(((df['ma120_temp']-df['ma120']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)
    
## 이동평균선 기준 scoring
## 향후 과제로 해당 weight를 찾는걸 해도 될듯.

    del(df['ma20_temp'])
    del(df['ma5_temp'])
    del(df['ma60_temp'])
    del(df['ma120_temp'])
    
    

    return df

In [38]:
page_number_lst = []  
for i in range(6):     # 원하는 페이지수만큼 range의 파라미터 설정 
    page_number_lst.append(i+1)

# print(code[1],code[2]) #code[2] : 코드번호 / code[1] : 회사명
for code_idx, code in df_code.iterrows():
    df=pd.DataFrame()
    list_date = []        # 리스트 초기화 
    list_endprice = []
    corp_total = []
    corp_name = []
    for _page in page_number_lst: # 1페이지부터 n페이지까지 크롤링  (Step1 참조.)

        url_head = 'https://finance.naver.com/item/sise_day.nhn?code={}&page={}'.format(code[2],_page)  # 종목코드와 페이지를 넣는 url 생성
        headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36'}
        web = requests.get(url_head, headers = headers).content #자료 요청
        soup = BeautifulSoup( web, 'html.parser') # html 파싱

        for tr in soup.find('table').find_all('tr'): # 일별시세 웹에서 ['날짜'.'종가','전일비'.. ] 행 크롤링, (table 태그안에 tr태그의 텍스트) 이미지참조.
            try:
                if len(tr.find_all("td")[0].get_text()) != 10: # 날짜가 비워진 셀은 패스 / 
                    pass
                else:
                    if tr.find_all("td")[0].get_text() in list_date and int(tr.find_all("td")[0].get_text()[:4]) < 2021: # 동일한 날짜가 기존에 있었는지 체크 (중복 페이지 예외처리)
                        break
                    else:
                        list_date.append(tr.find_all("td")[0].get_text())   # 날짜 데이터 삽입
                        list_endprice.append(tr.find_all('td')[1].get_text())  # 종가 데이터 삽입
                        corp_total.append(str(code[2]))  # 종목 코드 삽입
                        corp_name.append(str(code[1])) # 기업명 삽입
            except:
                pass
    for i in range(len(list_endprice)):
        list_endprice[i] = list_endprice[i].replace(",","")
        
    df['날짜'] = list_date               # 날짜 컬럼에 날짜 Valuse 삽입
    df['주식_종가'] = list_endprice      # 주식종가 컬럼에 종가 Valuse 삽입
    df['종목코드'] = corp_total          # 종목코드 컬럼에 코드 Valuse 삽입
    df['종목명'] = corp_name             # 종목명 컬럼에 기업명 Valuse 삽입
    
    globals()['df_{}'.format(str(code[2]))]= cross_score(df[df['종목코드']==str(code[2])])
    temp_df = globals()['df_{}'.format(str(code[2]))]
    df_total = pd.concat([df_total,temp_df],ignore_index=True, sort=True)
    df_list.append(['df_{}'.format(str(code[2]))])

    
    
print(time.time()-start)

1213.748429775238


<hr style="border-color:red; border-width:5px">

In [138]:
df_total

,날짜,종목명,종목코드,주식_종가,ma5_slope,ma20_slope,ma60_slope,증감,MA_score
0,2021.06.23,마이크로컨텍솔,098120,7270,16.0,48.5,6376.166667,-320,342.133333
1,2021.06.22,마이크로컨텍솔,098120,7590,-54.0,63.0,-0.000000,-100,-22.050000
2,2021.06.21,마이크로컨텍솔,098120,7690,-128.0,68.5,-0.000000,270,-72.475000
3,2021.06.18,마이크로컨텍솔,098120,7420,-42.0,50.5,-0.000000,-50,-16.775000
4,2021.06.17,마이크로컨텍솔,098120,7470,-52.0,62.5,-0.000000,120,-20.775000
5,2021.06.16,마이크로컨텍솔,098120,7350,-6.0,72.5,-0.000000,30,13.925000
6,2021.06.15,마이크로컨텍솔,098120,7320,126.0,89.5,-0.000000,270,110.575000
7,2021.06.14,마이크로컨텍솔,098120,7050,26.0,73.5,-0.000000,-160,36.575000
8,2021.06.11,마이크로컨텍솔,098120,7210,54.0,91.0,-0.000000,0,60.550000
9,2021.06.10,마이크로컨텍솔,098120,7210,70.0,81.0,-0.000000,-110,69.250000


In [137]:
df_total=df_total[['날짜','종목명','종목코드','주식_종가'\
                  ,'ma5','ma5_slope','ma20','ma20_slope',\
                 'ma60','ma60_slope']]

KeyError: "['ma5', 'ma20', 'ma60'] not in index"

In [69]:
# df_total['ma5_slope'] = df_total['ma5_slope'] *(-1)
# df_total['ma20_slope'] = df_total['ma20_slope'] *(-1)
# df_total['ma60_slope'] = df_total['ma60_slope'] *(-1)

In [71]:
# 1일 ->01일
month_ =''
day_ = ''
if len(str(datetime.today().month)) ==1 :
    month_ = str('0'+str(datetime.today().month))
else:
    pass
if len(str(datetime.today().day)) == 1:
    day_ = str('0'+str(datetime.today().day))
else:
    day_=str(datetime.today().day)

today_ = str(datetime.today().year)+'.'+str(month_)+'.'+str(day_)

In [93]:
df_final=df_total
del df_final['ma5']
del df_final['ma20']
del df_final['ma60']

In [95]:
df_final['증감']=0
try:
    for i in range(len(df_final['종목코드'])):
        if df_final['종목코드'][i]==df_final['종목코드'][i+1]:
            df_final.iloc[i,7]=\
            int(df_final.iloc[i,3])-int(df_final.iloc[i+1,3])
        else : pass

except KeyError:
    print("끝")

KeyboardInterrupt: 

In [136]:
df_final['MA_score'] = 0.7*df_final['ma5_slope'] +\
                    0.25*df_final['ma20_slope'] +\
                    0.05*df_final['ma60_slope']
df_final

,날짜,종목명,종목코드,주식_종가,ma5_slope,ma20_slope,ma60_slope,증감,MA_score
0,2021.06.23,마이크로컨텍솔,098120,7270,16.0,48.5,6376.166667,-320,342.133333
1,2021.06.22,마이크로컨텍솔,098120,7590,-54.0,63.0,-0.000000,-100,-22.050000
2,2021.06.21,마이크로컨텍솔,098120,7690,-128.0,68.5,-0.000000,270,-72.475000
3,2021.06.18,마이크로컨텍솔,098120,7420,-42.0,50.5,-0.000000,-50,-16.775000
4,2021.06.17,마이크로컨텍솔,098120,7470,-52.0,62.5,-0.000000,120,-20.775000
5,2021.06.16,마이크로컨텍솔,098120,7350,-6.0,72.5,-0.000000,30,13.925000
6,2021.06.15,마이크로컨텍솔,098120,7320,126.0,89.5,-0.000000,270,110.575000
7,2021.06.14,마이크로컨텍솔,098120,7050,26.0,73.5,-0.000000,-160,36.575000
8,2021.06.11,마이크로컨텍솔,098120,7210,54.0,91.0,-0.000000,0,60.550000
9,2021.06.10,마이크로컨텍솔,098120,7210,70.0,81.0,-0.000000,-110,69.250000


In [184]:
df_today = df_final[df_final['날짜']==today_]
for row_idx, row in df_today.iterrows():
    row[3] = int(row[3])

df_today['주식_종가'] = \
df_today['주식_종가'].apply(pd.to_numeric)



C:\Users\RYU\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [185]:
# df_today= \
# df_today[['날짜','종목명','종목코드','주식_종가','증감','MA_score']]
df_today['MA_score_per_주식_종가'] = df_today['MA_score']/df_today['주식_종가']

C:\Users\RYU\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [139]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [187]:
x = df_today[['MA_score_per_주식_종가','ma5_slope', 'ma20_slope']]
y = df_today[['증감']]

In [188]:
x_train, x_test, y_train, y_test = \
train_test_split(x, y, train_size=0.8, test_size=0.2)
line_fitter = LinearRegression()
line_fitter.fit(x_train, y_train )
print("coef : ",line_fitter.coef_)
print("intercept : ",line_fitter.intercept_)
print("R-squared : ", line_fitter.score(x_train, y_train))

coef :  [[ 2.37227805e+02 -9.05919876e-01 -2.34738793e-01]]
intercept :  [-39.75438984]
R-squared :  0.2865838551555928


In [191]:
import statsmodels.api as sm
from scipy import stats
est=sm.OLS(y,x)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                     증감   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.224
Method:                 Least Squares   F-statistic:                     258.1
Date:                Wed, 23 Jun 2021   Prob (F-statistic):          4.38e-147
Time:                        22:41:05   Log-Likelihood:                -22758.
No. Observations:                2675   AIC:                         4.552e+04
Df Residuals:                    2672   BIC:                         4.554e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
MA_score_per_주식_종가   350.4458    447